In [36]:
from datetime import datetime
date_today = datetime.today().strftime('%Y-%m-%d')

year = date_today[0:4]
month = date_today[5:7]
day = date_today[8:10]

print(date_today)

2024-08-19


In [49]:
import json
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import io
import boto3
from PIL import Image
from io import BytesIO
from sagemaker.s3 import S3Downloader


s3 = boto3.resource('s3')
my_bucket = s3.Bucket('ball-detection-batch-transform-2024')



file_key = f's3://ball-detection-batch-transform-2024/cleaned-jsons/{year}/{month}/{day}/{year}_{month}_{day}.json'
output = S3Downloader.read_file(file_key)
detections = json.loads(output)
for i in detections:
    image_path_s3 = f'images/{year}/{month}/{day}/{i[24:-4]}'
    object = my_bucket.Object(image_path_s3)
    response = object.get()
    file_stream = response['Body']
    img = Image.open(file_stream)
    width, height = img.size
    plt.imshow(img)
    detections_in_image = []
    data = detections[i]
    print(data)
    colors = dict()
    for j in data:
        detections_in_image.append(j)
    for k in detections_in_image:
        klass, score, x0, y0, x1, y1 = k
        if score < 0.25:
            continue
        cls_id = int(klass)
        if cls_id not in colors:
            colors[cls_id] = (random.random(), random.random(), random.random(), random.random())
        xmin = int(x0 * width)
        ymin = int(y0 * height)
        xmax = int(x1 * width)
        ymax = int(y1 * height)

        rect = plt.Rectangle(
            (xmin, ymin),
            xmax - xmin,
            ymax - ymin,
            fill = False,
            edgecolor = colors[cls_id],
            linewidth = 3.5
        )
        plt.gca().add_patch(rect)
        plt.gca().text(
            xmin,
            ymin - 2,
            "{:.3f}".format(score),
            bbox = dict(facecolor = colors[cls_id], alpha = 0.5),
            fontsize = 12,
            color = "white",
        )
        
        image_data = my_bucket.Object(f"output-images/{year}/{month}/{day}/{i[24:-4]}")
        file_stream = BytesIO()
        plt.savefig(file_stream, format = 'jpeg')
        image_data.put(Body = file_stream.getvalue())
        #plt.show()
        
        plt.close()

[[0.0, 0.5377199053764343, 0.7697994709014893, 0.8475220203399658, 0.8447315692901611, 0.9602493047714233]]
[[0.0, 0.700520932674408, 0.4252696633338928, 0.4790937900543213, 0.5755687355995178, 0.5730847120285034]]
[[0.0, 0.5751278400421143, 0.44395264983177185, 0.44661200046539307, 0.5502703785896301, 0.5239577293395996]]
[[0.0, 0.35933157801628113, 0.62443608045578, 0.4811268448829651, 0.8227110505104065, 0.7710930705070496]]
